## Helper

In [1]:
from manim import *
import numpy as np
from manim_slides import Slide

### Tex Preamble

In [2]:
# Preamble
preamble_path = "/home/ronakr/Documents/inplaceoracles/Inverting-a-Permutation-with-an-In-Place-Oracle/preamble.tex"
with open(preamble_path, 'r') as file:
    preamble = file.read()
myTemplate = TexTemplate()
myTemplate.add_to_preamble(preamble)

TexTemplate(_body='', tex_compiler='latex', description='', output_format='.dvi', documentclass='\\documentclass[preview]{standalone}', preamble='\\usepackage[english]{babel}\n\\usepackage{amsmath}\n\\usepackage{amssymb}\n\\usepackage[utf8]{inputenc}\n\n\\usepackage[letterpaper,margin=1in]{geometry}\n\\usepackage[OT1]{fontenc}\n\n\\usepackage{amsmath,amssymb,amsthm,amsfonts,latexsym,bbm,xspace,thm-restate}\n\\usepackage{graphicx,float,mathtools,braket,slantsc}\n\\usepackage{enumitem,booktabs,forest,mathdots,soul}\n\\usepackage[useregional]{datetime2}\n\\DTMusemodule{english}{en-US}\n\n\\usepackage{qcircuit,tikz}\n\n\\usepackage[colorlinks,citecolor=blue,,linkcolor=magenta,bookmarks=true]{hyperref}\n\\usepackage[capitalise]{cleveref}\n\n\\usepackage{multicol,array}\n\\usepackage{lipsum,framed}\n\n\\newtheorem{theorem}{Theorem}\n\\newtheorem*{theorem*}{Theorem}\n\\newtheorem{lemma}[theorem]{Lemma}\n\\newtheorem{claim}[theorem]{Claim}\n\\newtheorem{proposition}[theorem]{Proposition}\n\\ne

In [131]:
def ShowTexIndices(self, tex):
    # Observe first level labels
    tex_ = tex.copy().next_to(tex, DOWN)
    self.add(index_labels(tex_))

    # Observe second level labels
    tex__ = tex_.copy().next_to(tex_, DOWN)
    for part in tex__:
        self.add(index_labels(part))

### FlipTransform

In [ ]:
def FlipTransform(self, mobject_from=None, mobject_to=None):
    self.play(Homotopy(lambda x,y,z,t : (np.cos(t*np.pi/2)*x, y + 0.1*x*np.sin(t*np.pi/2), z), mobject_from, rate_func=rush_into, run_time=0.8))
    self.remove(mobject_from)
    self.play(Homotopy(lambda x,y,z,t : (np.sin(t*np.pi/2)*x, y - 0.1*x*np.cos(t*np.pi/2), z), mobject_to, rate_func=rush_from, run_time=0.8))

In [4]:
# def _flip(x,y,z,t):
#     return (np.cos(t*np.pi)*x, 0.5*x*np.sin(t*np.pi)+y, z)

# def Flip(mobject=None, *vargs, **kwargs):
#     return Homotopy(_flip, mobject, *vargs, **kwargs)

### GroverQuery

In [ ]:
def GroverQuery(self, bar_chart):
    def _negate(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*y*np.sin(t*np.pi), np.cos(t*np.pi)*y)
    idx = 0 # index of bar of marked element
    bar = bar_chart.bars[idx]
    self.play(Homotopy(_negate, bar, rate_func=rush_into, run_time=0.8))
    bar_chart.bars.remove(bar)
    bar_chart.bars.insert(idx, bar_chart._create_bar(idx, -bar_chart.values[idx]))
    bar_chart._update_colors()
    bar_chart.values[0] *= -1

### GroverDiffuseTransform

In [ ]:
def GroverDiffuseTransform(self, bar_chart, new_values):
    avg = np.average(bar_chart.values)
    def _diffuse_from(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*(y-avg)*np.sin(t*np.pi/2), np.cos(t*np.pi/2)*(y-avg) + avg)
    def _diffuse_to(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x + 0.3*(y-avg)*np.cos(t*np.pi/2), np.sin(t*np.pi/2)*(y-avg) + avg)
    
    self.play(*[Homotopy(_diffuse_from, bar, rate_func=rush_into, run_time=0.8) for bar in bar_chart.bars])
    for i,bar in enumerate(bar_chart.bars):
        bar_chart.bars.remove(bar)
        bar_chart.bars.insert(i, bar_chart._create_bar(i, new_values[i]))
    bar_chart._update_colors()
    bar_chart.values[:len(new_values)] = new_values
    self.play(*[Homotopy(_diffuse_to, bar, rate_func=rush_from, run_time=0.8) for bar in bar_chart.bars])

### Circuit Braces

In [5]:
def CircuitBraces(ckt, left, right):
    bL = Brace(ckt, LEFT)
    bLt = bL.get_tex(left + r" \rightarrow")
    bR = Brace(ckt, RIGHT)
    bRt = bR.get_tex(r"\rightarrow " + right)
    return VGroup(bL, bLt, bR, bRt)

## Sanity Check

In [ ]:
%%manim -ql SanityCheck

class SanityCheck(Scene):
    def construct(self):
        s0 = MathTex(r"a")
        s1 = MathTex(r"\frac{a}{b}")
        s2 = MathTex(r"\frac{b}{a}")
        tms0 = TransformMatchingShapes(s0,s1, key_map={-6351953992051140758:8757917165807693890})
        tms = TransformMatchingShapes(s1,s2)#, key_map={-6351953992051140758:8757917165807693890})
        # s1a = tms.get_mobject_key(tms.get_mobject_parts(s1)[0])
        # s2l = tms.get_mobject_key(tms.get_mobject_parts(s2)[1])
        # tms.key_map = {s1a:s2l}
        print(tms0.get_mobject_parts(s0))
        print(tms0.get_mobject_parts(s1))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s0)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[1]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[2]))

        print(tms.get_mobject_parts(s1))
        print(tms.get_mobject_parts(s2))

        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[2]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[2]))
        self.play(tms0)
        self.play(tms)

## Overview

### Query Complexity (motivation)
- What is a problem? 
  - Input: some f, Output: some property of f. 
  - Egs: 
    - (1) Given: N-bit integer. Output: is it nonzero? (show arrow to "integer", saying f : [N] -> {0,1}, transform) 
      (show after (2)) Given: function f : [N] -> {0,1}. Output: is there an x such that f(x) = 1?
      mention this is "searching for a 1"
    - (2) Given: a shuffled deck of cards labeled {1,...,N}. Output: location of the "1" card.
      Given permutation f: [N] -> [N]. Output: x such that f(x) = 1.
- "In Complexity theory we often care about how much time the best algorithm to solve a problem would take as the size of the input (N) increases"
- "That's really hard to lower bound. The best technique we have is query complexity."
- In query cxty, we imagine the input is behind a paywall, and we need to pay an oracle every time we want to access bits of it. (oracle pic)
  - I will tell the oracle x and the oracle will tell me f(x), each request is called a query
  <!-- the label of the xth card in the deck. (exchange) -->
  - It's fairly intuitive that, for these specific problems above, N queries are necessary classically, (implying an Omega(N) lower bound on time?).
  - How do we do quantum query complexity? 
    - Our first thought might be |x> -> |f(x)>. 
      - Violates QM if f is not reversible. 
    - Solution: |x>|a> -> |x>|a + f(x)>. -self inverse -equiv to phase oracles
    - Okay, but what if f is a permutation? 
    - Then |x> -> |f(x)> is fine, called in-place queries.

### Key Question of this talk
- Given a permutation f, how do XOR queries and in-place queries to f compare?
  
  <!-- however, the ability to make multiple queries in superposition lets us find "1" with O(sqrt(N)) queries, through what's called Grover's algorithm
  - Understanding this alg is crucial to understanding our alg, so let's briefly review -->

### Table of comparisons
- If we have access to both f and f^{-1}, then both models can simulate the other
- Omega(sqrt(N)) XOR queries required to exactly simulate an in-place query
  - In this paper: Omega(sqrt(N)) in-place to XOR
- IndexErasure (state conversion) O(sqrt(N)) vs O(1)
  - states of the form |x>|f(x)> -> |0>|f(x)>
- SetComp (promise decision problem) O(N^{1/7}) vs O(1)
  - venn diagram highlighting: two large sets are the same or have large symmetric difference
- Unstructured search (search problem) O(sqrt(N)) vs conjectured Omega(N). (cross out conj, write ours) In this paper: O(sqrt(N)).
  - N shuffled cards, locate "1"
- Trashy Simon's (promise search problem) O(log(N)) vs conjectured Omega(sqrt(N)) ### TODO: verify sqrt(N) correct!

### Grover's
- Our problem: Given a shuffled deck of cards labeled [N], find 1. 
- (Show 12345 example truth table with f, circle inv of 1) 
- (transform problem) given a permutation f : [N] -> [N], find x^* := f^{-1}(1).
- (disappear truth table)
- In Grover's algorithm, we start off with a uniform superposition over all x in [N] (bar chart)
- Repeatedly alternate between the following two operations:
  - Query, which negates the amplitude on x^* (flip)
  - Grover's Diffusion operator, which flips all amplitudes about their average (draw line, flip)
  - (quickly evolve a few times until x^* peaks) after ~sqrt N iterations of this, we will have made ~sqrt N queries and the probability of seeing x^* when we measure.

### Where Grover's fails
- key step is being able to take |x> -> (-1)^{f(x) = 1} |x>
- |x>|0> -> |x>|f(x)> -> (-1)^{f(x) = 1}|x>|f(x)> (->) |x>|0>  <- (new slide) function erasure is hard
- Without being able to erase f, interference fails

### In fact, we show func erasure is hard!
- pf omitted here.

### Our Alg
- Do the math, have a box reminding people what in-place and XOR do
- |x> -> |x>|x>|0> -> |x>|f(x)>|0> -> |x*>|1>|1> + sum|x>|f(x)>|0>
- -> |x*>|1>|1> + sum|f(x)>|f(x)>|0> -> |x*>|1>|1> + sum|f(x)>|0>|0>  -> |x*>|1>|1> + sum|f(x)>|1>|0> -> |x*>|1> + sum|f(x)>|0>
- convert to bars

### Step Thru Our Alg
- show bars

### Kashefi
- 
- Key fact: perm inv LBs for both Sf [Ambainis] and Pf [FK18]

### Kashefi Pfs (3 slides)
- Inverting circuits
- Kashefi arg (|x>|a> -[Sf]- |x>|f(x)> in corner so we can flip it to drive home that Sf inv = Sf)
- Our arg (|x> -[Pf]- |f(x)> in corner so we can flip it to drive home that Pf inv = P{f_inv})
- => function erasure is hard (analog of index erasure)

### Sf Pf box
- Grovers + Kashefi => Theta(sqrt(N)) XOR to in-place
- Our Alg + our arg => Theta(sqrt(N)) in-place to XOR

### Summary
- Unstructured Search takes Theta(sqrt(N)) queries (in-place or XOR) (algorithm)
- Function Erasure takes O(1) XOR Queries and O(sqrt(N)) in-place queries (kashefi)
- More generally, Sf Pf Sfinv Pfinv all req sqrt(N) queries to simulate each other
- Future work: decision problem separations
  - candidates: simon, emb perm inv


## TitleSlide

In [122]:
%%manim -qm TitleSlide1

class TitleSlide1(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        title = Tex(r"Quantum Search with In-Place Queries").scale(1.5).shift(1.5*UP)
        subtitle = Tex(r"Blake Holman, \textbf{Ronak Ramachandran}, and Justin Yirka").scale(0.75)
        subtitle.next_to(title, DOWN, buff=0.5)
        ronak = ImageMobject("ronak.png").scale_to_fit_height(2).next_to(subtitle, DOWN, buff=0.5)
        blake = ImageMobject("blake.jpg").scale_to_fit_height(2).next_to(ronak, LEFT, buff=1.5)
        justin = ImageMobject("jirka.jpg").scale_to_fit_height(2).next_to(ronak, RIGHT, buff=1.5)
        self.play(Add(title), Write(subtitle), Succession(FadeIn(blake), FadeIn(ronak), FadeIn(justin), run_time=1.8))
        self.next_slide()

Manim Community v0.19.0

Animation 0: Add(Tex('Quantum Search with In-Place Queries')), etc.:   0%|          | 0/60 [00:00<?, ?it/s]/home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py:1533: RuntimeWarning: invalid value encountered in scalar divide
  alpha = t / animation.run_time
Animation 0: Add(Tex('Quantum Search with In-Place Queries')), etc.:   2%|▏         | 1/60 [00:00<00:11,  5.28it/s]/home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py:1533: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = t / animation.run_time


[07/07/25 20:47:01] INFO     Animation 0 : Partial movie file written in                   ]8;id=596847;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=902892;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             720p30/partial_movie_files/TitleSlide1/3138625091_3416734514_                         
                             2358810818.mp4'                                                                       

[07/07/25 20:47:03] INFO     Animation 1 : Partial movie file written in                   ]8;id=314223;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=112541;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             720p30/partial_movie_files/TitleSlide1/4267955454_1443166991_                         
                             1301940517.mp4'                                                                       

                    INFO     Combining to Movie file.                                      ]8;id=170375;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=764529;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

...

                    INFO     Rendered TitleSlide1                                                      ]8;id=733245;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=433712;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[07/07/25 20:47:04] INFO     Generated 1 slides to                                                      ]8;id=771520;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=684004;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide1'                         

[07/07/25 20:47:04] INFO     Generated 1 slides to                                                      ]8;id=576148;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=614596;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide1'                         

                    INFO     Slide 'TitleSlide1' configuration written in                               ]8;id=835263;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=148324;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide1.json'                          

                    INFO     Slide 'TitleSlide1' configuration written in                               ]8;id=189888;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=176054;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide1.json'                          

## Key Question

In [106]:
%%manim -ql KeyQuestion

class KeyQuestion(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        question = Tex(
            r"Given a \textbf{permutation} $f$, how do \textbf{XOR queries} \\ and \textbf{in-place queries} to $f$ compare?",
            substrings_to_isolate=[r"\textbf{XOR queries}", r"\textbf{in-place queries}"]
        )
        question.set_color_by_tex(r"XOR", XOR)
        question.set_color_by_tex(r"in-place", IN_PLACE)
        self.play(Write(question))
        self.next_slide()

Manim Community v0.19.0

[07/07/25 20:21:01] INFO     Animation 0 : Partial movie file written in                   ]8;id=880777;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=376679;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/KeyQuestion/1248406331_4213525962_                         
                             223132457.mp4'                                                                        

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=871783;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=608042;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/KeyQuestion/3564501620_1443166991_                         
                             2560561030.mp4'                                                                       

                    INFO     Combining to Movie file.                                      ]8;id=346303;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=410278;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=113916;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=334176;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/KeyQuestion.mp4'                                                               
                                                                                                                   

                    INFO     Rendered KeyQuestion                                                      ]8;id=64303;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=614074;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[07/07/25 20:21:02] INFO     Generated 1 slides to                                                      ]8;id=302090;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=924300;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

[07/07/25 20:21:02] INFO     Generated 1 slides to                                                      ]8;id=785124;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=436834;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=268665;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=476520;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=396711;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=697168;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

## XORvsInPlace

In [ ]:
%%manim -ql XORvsInPlace

class XORvsInPlace(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        
        
        xor_bullets = [
            MathTex(r"\ket{x}\ket{a} \rightarrow \ket{x}\ket{f(x)}", tex_template=myTemplate),
            Text("Any f"),
            Text("Can erase f(x)"),
            Text("Self-inverse"),
            Text("Phase version")
        ]
        inplace_bullets = [
            MathTex(r"\ket{x} \rightarrow \ket{f(x)}", tex_template=myTemplate),
            Text("Bijections"),
            Text("Can erase x")
        ]
        
        xor_group = VGroup(*xor_bullets).arrange(DOWN)
        inplace_group = VGroup(*inplace_bullets).arrange(DOWN)

        tchart = MobjectTable(
            [[xor_group, inplace_group]],
            col_labels=[Text("XOR"), Text("In-Place")],
            arrange_in_grid_config={"row_alignments":'uu'})

        self.play(
            Write(tchart.get_horizontal_lines()), 
            Write(tchart.get_vertical_lines()),
            Write(tchart.get_col_labels()))
        self.next_slide()

        for i in range(len(xor_bullets)):
            self.play(Write(xor_bullets[i]))
            self.next_slide()

            if i < len(inplace_bullets):
                self.play(Write(inplace_bullets[i]))
                self.next_slide()

## Inverting Circuits

In [123]:
%%manim -ql InvertingCircuits1

class InvertingCircuits1(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{A} & \multigate{1}{C} & \gate{D} & \qw \\
            & \gate{B} & \ghost{C} & \gate{E} & \qw
        }'''
        abcde_ckt = MathTex(abcde_tex, tex_template=myTemplate)
        abcde_braces = CircuitBraces(abcde_ckt, r"|\psi\rangle", r"\mathcal{C}|\psi\rangle")
        abcde = VGroup(abcde_ckt, abcde_braces)

        self.play(Write(abcde))
        self.next_slide()

        inv_abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{D^{-1}} & \multigate{1}{C^{-1}} & \gate{A^{-1}} & \qw \\
            & \gate{E^{-1}} & \ghost{C^{-1}} & \gate{B^{-1}} & \qw
        }'''
        inv_abcde_ckt = MathTex(inv_abcde_tex, tex_template=myTemplate)
        inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"\mathcal{C} |\psi\rangle", r"|\psi\rangle")
        inv_abcde = VGroup(inv_abcde_ckt, inv_abcde_braces)

        FlipTransform(self, abcde, inv_abcde)
        self.next_slide()

        new_inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"|\psi\rangle", r"\mathcal{C}^{-1} |\psi\rangle")
        self.play(Transform(inv_abcde_braces, new_inv_abcde_braces))
        # self.play(*[TransformMatchingShapes(inv_abcde_braces[i], new_inv_abcde_braces[i]) for i in (1,3)])
        self.next_slide()
        

Manim Community v0.19.0

[07/07/25 20:48:28] INFO     Animation 0 : Using cached data (hash :                           ]8;id=207348;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=929362;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1248406331_3362818412_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=599513;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=207022;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_1443166991_2630592056)                                                     

[07/07/25 20:48:29] INFO     Animation 2 : Using cached data (hash :                           ]8;id=911874;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=890232;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_4174954566_2375220172)                                                     

[07/07/25 20:48:30] INFO     Animation 3 : Partial movie file written in                   ]8;id=251536;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=181720;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/InvertingCircuits1/3564501620_3979                         
                             586510_1101682277.mp4'                                                                

...

                    INFO     Rendered InvertingCircuits1                                               ]8;id=660627;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=763110;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 7 animations                                                                   

[07/07/25 20:48:32] INFO     Generated 3 slides to                                                      ]8;id=320690;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=955021;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

[07/07/25 20:48:32] INFO     Generated 3 slides to                                                      ]8;id=166827;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=778316;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=682972;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=222824;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=839187;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=36152;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

## Kashefi

### OracleCircuit

In [6]:
def OracleCircuit(oracle_tex, left, right, length_tex=r"o(\sqrt{N})", flipped=False):
    flipped = -1 if flipped else 1
    oracle_color, ckt_color = ((IN_PLACE, XOR) if oracle_tex[0] == 'P' else (XOR, IN_PLACE))

    oracle_mathtex = MathTex(oracle_tex)
    oracle_box = SurroundingRectangle(oracle_mathtex, buff=0.2, color=WHITE, fill_opacity=1, fill_color=oracle_color)
    oracle_drawing = VGroup(oracle_box, oracle_mathtex)

    ckt_outline = RoundedRectangle(corner_radius=0.25, height=4.0, width=6.5, fill_opacity=1, fill_color=ckt_color)

    ckt_input = MathTex(left + r" \rightarrow").next_to(ckt_outline, LEFT)
    ckt_output = MathTex(r"\rightarrow " + right).next_to(ckt_outline, RIGHT)
    ckt_input_and_output = VGroup(ckt_input, ckt_output)

    len_line = NumberLine(length=ckt_outline.width, x_range=[0, 1, 1]).next_to(ckt_outline, UP)
    len_label = MathTex(length_tex).next_to(len_line, UP)
    ckt_length = VGroup(len_line, len_label)
    # DoubleArrow(pt1, pt2, tip_shape_end=LineArrowTip, tip_shape_start=LineArrowTip, tip_length=0.1).next_to(ckt_outline, UP)
    
    oracles = VGroup()
    oracles.add(oracle_drawing.move_to(ckt_outline.get_center()).shift(flipped*2.3*LEFT + UP))
    oracles.add(oracles[0].copy().shift(flipped*1.5*RIGHT + DOWN))
    oracles.add(oracles[0].copy().shift(flipped*3*RIGHT))
    oracles.add(oracles[0].copy().shift(flipped*4.5*RIGHT + 2*DOWN))

    ckt = VGroup(ckt_outline, ckt_input_and_output, ckt_length, oracles)

    return ckt

### Kashefi

In [7]:
%%manim -ql Kashefi

class Kashefi(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        ckt = OracleCircuit(r"S_f", r"|\psi\rangle", r"P_f|\psi\rangle")
        ckt_f = OracleCircuit(r"S_f", r"|x\rangle", r"|f(x)\rangle")

        ckt_flipped = OracleCircuit(r"S_f^{-1}", r"|f(x)\rangle", r"|x\rangle", flipped=True)
        flipped_ckts = [ckt_flipped]
        flipped_ckts.append(OracleCircuit(r"S_f^{-1}", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"S_f", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        
        self.play(FadeIn(ckt[0]), FadeIn(ckt[1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckt[-1]], run_time=1.2), FadeIn(ckt[2], run_time=1.2))
        self.next_slide()
        self.play(ReplacementTransform(ckt, ckt_f))
        self.next_slide()

        FlipTransform(self, ckt_f, ckt_flipped)
        self.next_slide()
        for i in range(len(flipped_ckts) - 1):
            self.play(ReplacementTransform(flipped_ckts[i], flipped_ckts[i+1]))
            self.next_slide()

        contradiction = Tex(r"Violates $\Omega(\sqrt{N})$ lower bound for computing $f^{-1}$ using $S_f$").next_to(flipped_ckts[-1][0], DOWN)
        self.play(Write(contradiction))
        self.next_slide()

Manim Community v0.19.0

[07/06/25 22:04:44] INFO     Animation 0 : Partial movie file written in                   ]8;id=281524;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=981343;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/1248406331_1603910194_2231                         
                             32457.mp4'                                                                            

[07/06/25 22:04:45] INFO     Animation 1 : Partial movie file written in                   ]8;id=198289;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=624353;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/3564501620_1443166991_2892                         
                             6024.mp4'                                                                             

[07/06/25 22:04:47] INFO     Animation 2 : Partial movie file written in                   ]8;id=755832;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=451585;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/3564501620_3827268052_3467                         
                             307692.mp4'                                                                           

[07/06/25 22:04:49] INFO     Animation 3 : Partial movie file written in                   ]8;id=217529;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=74454;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/3564501620_1443166991_4270                         
                             728721.mp4'                                                                           

...

                    INFO     Rendered Kashefi                                                          ]8;id=27518;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=526953;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 15 animations                                                                  

[07/06/25 22:05:31] INFO     Generated 7 slides to                                                      ]8;id=240389;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=307223;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

[07/06/25 22:05:31] INFO     Generated 7 slides to                                                      ]8;id=67078;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=258834;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=229586;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=115176;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=274682;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=891805;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

## Our LB

In [8]:
%%manim -ql OurLB

class OurLB(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        ckt = OracleCircuit(r"P_f", r"|\psi\rangle", r"S_f|\psi\rangle")

        ckt_flipped = OracleCircuit(r"P_f^{-1}", r"S_f|\psi\rangle", r"|\psi\rangle", flipped=True)
        flipped_ckts = [ckt_flipped]
        flipped_ckts.append(OracleCircuit(r"P_f^{-1}", r"|\psi\rangle", r"S_f^{-1}|\psi\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"P_f^{-1}", r"|\psi\rangle", r"S_f|\psi\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"P_f^{-1}", r"|x\rangle|0\rangle", r"|x\rangle|f(x)\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"P_f", r"|x\rangle|0\rangle", r"|x\rangle|f^{-1}(x)\rangle", flipped=True))
        
        self.play(FadeIn(ckt[0]), FadeIn(ckt[1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckt[-1]], run_time=1.2), FadeIn(ckt[2], run_time=1.2))
        self.next_slide()

        FlipTransform(self, ckt, ckt_flipped)
        self.next_slide()
        for i in range(len(flipped_ckts) - 1):
            self.play(ReplacementTransform(flipped_ckts[i], flipped_ckts[i+1]))
            self.next_slide()

        contradiction = Tex(r"Violates $\Omega(\sqrt{N})$ lower bound for computing $f^{-1}$ using $P_f$").next_to(flipped_ckts[-1][0], DOWN)
        self.play(Write(contradiction))
        self.next_slide()

Manim Community v0.19.0

[07/06/25 22:06:03] INFO     Animation 0 : Partial movie file written in                   ]8;id=250863;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=797644;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/1248406331_1255663270_223132                         
                             457.mp4'                                                                              

[07/06/25 22:06:04] INFO     Animation 1 : Partial movie file written in                   ]8;id=452105;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=41774;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/3564501620_1443166991_272226                         
                             6963.mp4'                                                                             

[07/06/25 22:06:05] INFO     Animation 2 : Partial movie file written in                   ]8;id=833910;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=743750;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/3564501620_2153506756_907196                         
                             374.mp4'                                                                              

[07/06/25 22:06:07] INFO     Animation 3 : Partial movie file written in                   ]8;id=420465;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=294160;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/3564501620_1443166991_344077                         
                             7010.mp4'                                                                             

...

                    INFO     Rendered OurLB                                                            ]8;id=785192;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=639945;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 17 animations                                                                  

[07/06/25 22:06:50] INFO     Generated 8 slides to                                                      ]8;id=702931;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=645314;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

[07/06/25 22:06:50] INFO     Generated 8 slides to                                                      ]8;id=459172;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=508775;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

[07/06/25 22:06:51] INFO     Slide 'OurLB' configuration written in                                     ]8;id=603003;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=241917;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

[07/06/25 22:06:51] INFO     Slide 'OurLB' configuration written in                                     ]8;id=673733;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=119463;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

## AlgCkt

In [70]:
%%manim -ql AlgCircuit

class AlgCircuit(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        # qcircuit
        qcircuit = r'''
        \Qcircuit @C=.5em @R=0.5em @!R {
         & & & & & & \mbox{\hspace{1.8em}\textit{Mark}} & & & & & & & & \mbox{\textit{Shift}} & & & & & & & & \mbox{\textit{Diffuse the Difference}} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \ctrl{4}  & \qw       & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \multigate{2}{\Ppi} & \ctrl{3}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \multigate{2}{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \ctrl{4}  & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \ctrl{2}  & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \rstick{\ket{\psi_{t}}_\regA} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \ctrl{4}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \qw       & \ctrl{1}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}
    \inputgroupv{2}{4}{0.5em}{2em}{\ket{\psi_{t-1}}_\regA \hspace{1.3em}} \\
    	& \lstick{\ket{0}_\regC \Big\{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ctrlo{-1} & \ctrlo{1} & \ctrlo{2} & \ctrlo{3} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \gate{H}  & \ctrl{-1} & \gate{H}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \meter    & \cw       & \rstick{\text{\textit{Measure} }\regC} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \targ     & \qw       & \qw       & \multigate{2}{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \rstick{\ket{0^n}_\regB} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}
    \inputgroupv{6}{8}{0.5em}{2em}{\ket{0^n}_\regB \hspace{0.5em}}
    \gategroup{2}{29}{4}{29}{0em}{\}}
    \gategroup{6}{21}{8}{21}{0em}{\}}
    \gategroup{2}{4}{8}{10}{0.7em}{--}
    \gategroup{2}{13}{8}{18}{0.7em}{--}
    \gategroup{2}{21}{5}{25}{0.7em}{--}\\
    }
        '''
        label = MathTex(qcircuit, tex_template=myTemplate).scale(0.5)
        self.play(Write(label))
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[07/06/25 15:26:16] INFO     Animation 0 : Partial movie file written in                   ]8;id=749339;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=761827;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/AlgCircuit/1248406331_1682520363_2                         
                             23132457.mp4'                                                                         

[07/06/25 15:26:19] INFO     Animation 1 : Partial movie file written in                   ]8;id=495946;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=329100;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/AlgCircuit/3564501620_722531385_75                         
                             4778375.mp4'                                                                          

[07/06/25 15:26:22] INFO     Animation 2 : Partial movie file written in                   ]8;id=225440;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=659469;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/AlgCircuit/3564501620_1443166991_6                         
                             68892496.mp4'                                                                         

                    INFO     Combining to Movie file.                                      ]8;id=841673;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=423332;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

...

                    INFO     Rendered AlgCircuit                                                       ]8;id=163794;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=326977;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 3 animations                                                                   

[07/06/25 15:26:23] INFO     Generated 1 slides to                                                      ]8;id=245231;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=78076;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgCircuit'                          

[07/06/25 15:26:23] INFO     Generated 1 slides to                                                      ]8;id=544376;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=820859;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgCircuit'                          

                    INFO     Slide 'AlgCircuit' configuration written in                                ]8;id=139872;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=22977;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgCircuit.json'                           

                    INFO     Slide 'AlgCircuit' configuration written in                                ]8;id=551339;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=808781;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgCircuit.json'                           

## StepThruGroverBar

In [81]:
def get_Grover_amps(t, N=10):
    a_t = np.sin((2*t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((2*t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

In [103]:
%%manim -ql StepThruGroverBar

class StepThruGroverBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        N = 10
        T = int(np.ceil(np.pi/4 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)

        title = Tex("Grover's Algorithm").next_to(chart, UP)
        self.play(Write(title, run_time=0.5), DrawBorderThenFill(chart), DrawBorderThenFill(y_axis_label))
        self.next_slide()

        a_t,b_t = get_Grover_amps(0,N)
        for t in range(1,T):
            # Query
            GroverQuery(self, chart)
            self.next_slide()

            # Average
            avg_line = chart.plot(lambda x : np.average(chart.values), stroke_width=1)
            self.play(Create(avg_line))
            self.next_slide()

            # Diffuse
            a_t,b_t = get_Grover_amps(t,N)
            values = [a_t] + [b_t]*(N-1)
            GroverDiffuseTransform(self, chart, values)
            self.play(FadeOut(avg_line))
            self.next_slide()


Manim Community v0.19.0

[07/07/25 18:09:29] INFO     Animation 0 : Using cached data (hash :                           ]8;id=746423;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=314749;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1248406331_642536877_223132457)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=395446;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=85785;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_1443166991_3826376975)                                                     

[07/07/25 18:09:30] INFO     Animation 2 : Using cached data (hash :                           ]8;id=365389;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=806154;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_533182947_4175571009)                                                      

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=340059;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=410804;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_1443166991_252930429)                                                      

...

                    INFO     Rendered StepThruGroverBar                                                ]8;id=836792;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=616250;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[07/07/25 18:09:47] INFO     Generated 7 slides to                                                      ]8;id=116718;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=588090;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

[07/07/25 18:09:47] INFO     Generated 7 slides to                                                      ]8;id=957386;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=930005;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=149983;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=818899;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=781071;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=538916;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

## Where Grover Fails

In [159]:
%%manim -ql GroverBreakdown

class GroverBreakdown(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        query_tex_cases = MathTex(r"|x\rangle \rightarrow \begin{cases} -|x\rangle & \text{if $f(x) = 1$} \\ |x\rangle & \text{otherwise} \end{cases}")
        query_tex = MathTex(r"|x\rangle \rightarrow (-1)^{f(x)=1}|x\rangle")
        self.play(Write(query_tex_cases))
        self.next_slide()
        self.play(ReplacementTransform(query_tex_cases, query_tex))
        self.next_slide()

        self.play(query_tex.animate.shift(3*UP))
        steps = []
        steps.append(MathTex(
            r"|x\rangle",
            r"|0\rangle",
        ).shift(UP))
        steps.append(MathTex(
            r"\xrightarrow{S_f}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.append(MathTex(
            r"\xrightarrow{\phantom{S_f}} (-1)^{f(x)=1}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.append(MathTex(
            r"\xrightarrow{S_f} (-1)^{f(x)=1}",
            r"|x\rangle",
            r"|0\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps[0].align_to(steps[1][1], LEFT)
        steps[1][0][0:2].set_fill(color=XOR)
        steps[3][0][0:2].set_fill(color=XOR)

        # ShowTexIndices(self, steps)

        self.play(FadeIn(steps[0][0]))
        self.next_slide()
        self.play(FadeIn(steps[0][1]))
        self.next_slide()
        self.play(FadeIn(steps[1]))
        self.next_slide()
        self.play(FadeIn(steps[2]))
        self.next_slide()
        self.play(FadeIn(steps[3][0]), FadeIn(steps[3][1]), FadeIn(steps[3][2]))
        self.next_slide()
        self.play(FadeOut(steps[3][-1]))
        self.next_slide()

        self.play(Circumscribe(VGroup(steps[2], steps[3]), color=RED_E, fade_out=True))

        func_eras = MathTex(
            r"|x\rangle|f(x)\rangle",
            r"\rightarrow",
            r"|x\rangle",
            r"|0\rangle",
        )
        to_fade = [query_tex, steps[0], steps[1], steps[2][0]]
        self.play(
            FadeOut(*to_fade),
            ReplacementTransform(steps[2][1], func_eras[0]),
            ReplacementTransform(steps[3][0], func_eras[2]),
            ReplacementTransform(steps[3][1], func_eras[3]),
            FadeIn(func_eras[1])
        )
        self.next_slide()

        func_eras_title = Tex("Function Erasure").next_to(func_eras, 2*UP)
        self.play(Write(func_eras_title))
        self.next_slide()


Manim Community v0.19.0

[07/07/25 22:29:48] INFO     Animation 0 : Using cached data (hash :                           ]8;id=947144;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=721517;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1248406331_4084572644_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=251328;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=65251;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_1443166991_1727944622)                                                     

[07/07/25 22:29:49] INFO     Animation 2 : Using cached data (hash :                           ]8;id=6702;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=174994;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_2470910229_2002292858)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=982490;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=707529;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_1443166991_1304989367)                                                     

...

                    INFO     Rendered GroverBreakdown                                                  ]8;id=284148;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=332130;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 22 animations                                                                  

[07/07/25 22:29:56] INFO     Generated 10 slides to                                                     ]8;id=158891;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=60500;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

[07/07/25 22:29:56] INFO     Generated 10 slides to                                                     ]8;id=999677;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=532218;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=138549;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=263071;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=195439;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=468364;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

## StepThruOurAlgBar

In [6]:
def get_HRY_amps(t, N=10):
    a_t = np.sin((t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

In [ ]:
%%manim -ql StepThruOurAlgBar

class StepThruOurAlgBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        N = 10
        T = int(np.ceil(np.pi/2 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)

        self.play(DrawBorderThenFill(chart), DrawBorderThenFill(y_axis_label))
        self.next_slide()

        for t in range(1,T):
            a_t,b_t = get_HRY_amps(t,N)
            values = [a_t] + [b_t]*(N-1)
            self.play(chart.animate.change_bar_values(values))
            self.next_slide()


Manim Community v0.19.0

[07/07/25 10:23:48] INFO     Animation 0 : Using cached data (hash :                           ]8;id=253282;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=918419;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1248406331_372114813_223132457)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=523018;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=870411;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_1443166991_2066110908)                                                     

[07/07/25 10:23:49] INFO     Animation 2 : Using cached data (hash :                           ]8;id=611880;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=70994;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_2632030998_3878435577)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=906365;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=725376;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3564501620_1443166991_4276947414)                                                     

...

                    INFO     Rendered StepThruOurAlgBar                                                ]8;id=531437;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=59402;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 10 animations                                                                  

[07/07/25 10:23:51] INFO     Generated 5 slides to                                                      ]8;id=678110;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=581301;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruOurAlgBar'                   

                    INFO     Generated 5 slides to                                                      ]8;id=355354;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=545913;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruOurAlgBar'                   

                    INFO     Slide 'StepThruOurAlgBar' configuration written in                         ]8;id=623223;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=174752;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruOurAlgBar.json'                    

                    INFO     Slide 'StepThruOurAlgBar' configuration written in                         ]8;id=202358;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=725127;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruOurAlgBar.json'                    